In [ ]:
# @title Setup: install & imports
!pip install torch torchvision --quiet

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time
from typing import Tuple, List

import random

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Random seed set to {seed}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# @title Experiment config
USE_AUGMENTATION = True
BATCH_SIZE = 128
EPOCHS = 40
LEARNING_RATE = 0.1
WEIGHT_DECAY = 5e-4
MOMENTUM = 0.9

PRINT_EVERY = 100        # batches
NUM_WORKERS = 2          # increase if Colab runtime is beefy

print('Config:')
print(f' augmentation       : {USE_AUGMENTATION}')
print(f' batch size         : {BATCH_SIZE}')
print(f' epochs             : {EPOCHS}')
print(f' lr / momentum / wd : {LEARNING_RATE} / {MOMENTUM} / {WEIGHT_DECAY}')
print(f' device             : {device}')


In [ ]:
# @title Data pipeline (CIFAR-10)
cifar10_mean = (0.4914, 0.4822, 0.4465)
cifar10_std  = (0.2470, 0.2435, 0.2616)

if USE_AUGMENTATION:
    train_transform = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std),
    ])
else:
    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(cifar10_mean, cifar10_std),
    ])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_transform
)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

classes = train_dataset.classes
print('Classes:', classes)
print('Train batches:', len(train_loader), 'Test batches:', len(test_loader))


In [ ]:
# @title Plain Block




class PlainBlock(nn.Module):
    """
    Same internal conv/bn/relu structure but NO skip connection.
    We still allow stride to downsample, but we don't add x.
    """
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2   = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.conv2(out)
        out = self.bn2(out)

        out = F.relu(out, inplace=True)
        return out


In [ ]:
# @title Model definitions: PlainNet14CIFAR

def make_layer(block_cls, in_channels, out_channels, num_blocks, first_stride):
    """
    Helper to create a stage with `num_blocks` blocks.
    first block can downsample via stride=2.
    rest have stride=1.
    """
    layers = []
    strides = [first_stride] + [1]*(num_blocks-1)
    c_in = in_channels
    for stride in strides:
        layers.append(block_cls(c_in, out_channels, stride))
        c_in = out_channels
    return nn.Sequential(*layers), out_channels





class PlainNet14CIFAR(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1   = nn.BatchNorm2d(16)

        self.layer1, c1 = make_layer(PlainBlock, 16, 16, num_blocks=2, first_stride=1)
        self.layer2, c2 = make_layer(PlainBlock, c1, 32, num_blocks=2, first_stride=2)
        self.layer3, c3 = make_layer(PlainBlock, c2, 64, num_blocks=2, first_stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc      = nn.Linear(c3, num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


plain14  = PlainNet14CIFAR().to(device)


print('PlainNet14 params:', count_params(plain14))


In [ ]:
# @title Training / evaluation helpers

def train_one_epoch(model, loader, criterion, optimizer, epoch_idx=0):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    start_time = time.time()
    for batch_idx, (inputs, targets) in enumerate(loader):
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if (batch_idx + 1) % PRINT_EVERY == 0:
            print(f"[epoch {epoch_idx} | batch {batch_idx+1}/{len(loader)}] "
                  f"loss={loss.item():.4f} "
                  f"acc={(100.*correct/total):.2f}%")

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    epoch_time = time.time() - start_time
    return epoch_loss, epoch_acc, epoch_time


@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    for (inputs, targets) in loader:
        inputs, targets = inputs.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc


In [ ]:
# @title Train both models (ResNet14 vs PlainNet14)

def run_experiment(model, train_loader, test_loader, epochs, lr, wd, momentum, label='model'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(),
        lr=lr, momentum=momentum, weight_decay=wd
    )
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer,
        milestones=[int(epochs*0.5)],
        gamma=0.1
    )

    history = {
        'train_loss': [],
        'train_acc':  [],
        'test_loss':  [],
        'test_acc':   [],
        'epoch_time': [],
    }

    print(f"\n=== Training {label} ===")
    for epoch in range(1, epochs+1):
        train_loss, train_acc, train_time = train_one_epoch(
            model, train_loader, criterion, optimizer, epoch_idx=epoch
        )
        test_loss, test_acc = evaluate(model, test_loader, criterion)
        scheduler.step()

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['test_loss'].append(test_loss)
        history['test_acc'].append(test_acc)
        history['epoch_time'].append(train_time)

        print(f"[{label}] Epoch {epoch}/{epochs} "
              f"| train_acc={train_acc:.2f}% "
              f"| test_acc={test_acc:.2f}% "
              f"| train_loss={train_loss:.4f} "
              f"| test_loss={test_loss:.4f} "
              f"| time/epoch={train_time:.1f}s")

    return model, history

set_seed(42)

plain14  = PlainNet14CIFAR()


plain14, hist_plain = run_experiment(
    plain14, train_loader, test_loader,
    epochs=EPOCHS,
    lr=LEARNING_RATE,
    wd=WEIGHT_DECAY,
    momentum=MOMENTUM,
    label='PlainNet14'
)


In [ ]:
# @title Plot accuracy/loss comparison


plt.figure()
plt.plot(epochs_axis, hist_res['train_loss'], label='ResNet14 train')
plt.plot(epochs_axis, hist_res['test_loss'],  label='ResNet14 test')
plt.plot(epochs_axis, hist_plain['train_loss'], label='PlainNet14 train')
plt.plot(epochs_axis, hist_plain['test_loss'],  label='PlainNet14 test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# @title Final comparison summary
def summarize_hist(label, hist):
    best_test_acc = max(hist['test_acc'])
    last_test_acc = hist['test_acc'][-1]
    avg_epoch_time = sum(hist['epoch_time']) / len(hist['epoch_time'])
    print(f"{label}:")
    print(f"  best test acc : {best_test_acc:.2f}%")
    print(f"  last test acc : {last_test_acc:.2f}%")
    print(f"  avg epoch sec : {avg_epoch_time:.1f}s/epoch")
    print()


summarize_hist('PlainNet14', hist_plain)

print('Note:')
print('- ResNet14 uses skip connections (residual learning).')
print('- PlainNet14 is the same architecture but without the identity addition.')
print('- Difference in accuracy shows how residual connections help optimization,')
print('  especially as depth increases.')


In [ ]:
# @title Save checkpoints

torch.save(plain14.state_dict(), 'plainnet14_cifar10.pth')
